In [25]:
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal

from tqdm import tqdm_notebook as tqdm

In [4]:
data_fname = 'data.npz'
load_data = np.load(data_fname)
true_pi = load_data['pi']
true_mu = load_data['mu']
samples = load_data['samples']
adv_sample = load_data['adv_sample']

In [6]:
true_pi, true_mu, samples[::50], adv_sample

(array([0.23705428, 0.06574439, 0.39978303, 0.07123911, 0.22617919]),
 array([[2.02889543, 0.58387321],
        [1.48504397, 3.96130589],
        [1.26997612, 1.46258317],
        [3.4355634 , 1.58225174],
        [1.47788008, 3.48312234]]),
 array([[ 4.17993725,  1.24175583],
        [ 2.68098744,  0.32334823],
        [ 1.41416657,  0.3940763 ],
        [ 3.04086445,  4.25610681],
        [ 1.87104121,  4.03256269],
        [ 5.0214534 ,  1.57025624],
        [ 3.1324006 ,  1.80930315],
        [ 3.48725692,  2.19854219],
        [ 2.67629033, -0.01613154],
        [ 2.5919368 ,  2.00105177],
        [-0.10403355,  0.37719824],
        [ 3.65484623,  1.83349119],
        [ 0.27127926,  2.74596415],
        [ 1.16614525,  1.58661359],
        [ 1.47989679,  0.89424985],
        [ 0.9680544 ,  1.00286455],
        [ 4.67228469,  2.75282497],
        [ 0.3507703 ,  3.45548201],
        [ 2.11184957,  2.31217297],
        [ 2.10118491,  4.49285233]]),
 array([-1., -1.]))

In [32]:
def MoG_prob(x, pi, mu, cov):
    K, dim = mu.size()
    assert x.size() == (dim,)
    assert pi.size() == (K,)
    assert cov.size() == (K, dim, dim)
    
    prob = 0.0
    for k in range(K):
        log_prob_k = -dim * 0.5 * math.log(2 * math.pi) - 0.5 * cov[k].logdet() - 0.5 * cov[k].inverse().matmul(x - mu[k]).dot(x - mu[k])
        prob += torch.exp(log_prob_k) * pi[k] / pi.sum()
    return prob

def MoG_loss(X, z, pi, mu, cov, lam):
    # z: adv_sample
    K, dim = mu.size()
    assert X.size(1) == dim
    assert pi.size() == (K,)
    assert cov.size() == (K, dim, dim)
    
    loss = lam * MoG_prob(z, pi, mu, cov)
    for x in X:
        loss -= torch.log(MoG_prob(x, pi, mu, cov))
    return loss

def GD_solver(samples, adv_sample, lam=2.0, max_step=10000, lr=0.01, K=5, dim=2):
    X = torch.tensor(samples)
    z = torch.tensor(adv_sample)
    
    pi = torch.rand(K, requires_grad=True)
    pi /= pi.sum()
    mu = torch.randn(K, dim, requires_grad=True)
    cov = torch.eye(dim).repeat(K, 1).requires_grad_()
    
    print('*** Init ***')
    print('pi:')
    print(pi)
    print('mu:')
    print(mu)
    print('cov:')
    print(cov)
    print('loss:')
    print(MoG_loss(X, z, pi, mu, cov, lam=lam))
    
    optimizer = optim.SGD([pi, mu, cov], lr=lr)
    
    for step in tqdm(range(max_step)):
        optimizer.zero_grad()
        loss = MoG_loss(X, z, pi, mu, cov, lam=lam)
        loss.backward()
        optimizer.step()
        
        if (step + 1) % 100 == 0:
            print('Step {}'.format(step))
            print('pi:')
            print(pi)
            print('mu:')
            print(mu)
            print('cov:')
            print(cov)
            print('loss:')
            print(loss)
    
    return pi, mu, cov

In [34]:
lam=2.0
max_step=10000
lr=0.01
K=5
dim=2

X = torch.FloatTensor(samples[:10])
z = torch.FloatTensor(adv_sample)

pi = torch.rand(K, dtype=torch.float)
pi /= pi.sum()
pi.requires_grad_()
mu = torch.randn(K, dim, dtype=torch.float)
mu.requires_grad_()
cov = torch.eye(dim, dtype=torch.float).repeat(K, 1, 1).requires_grad_()

## @TODO: collect parameters & named parameters; register hook

print('*** Init ***')
print('pi:')
print(pi)
print('mu:')
print(mu)
print('cov:')
print(cov)
print('loss:')
print(MoG_loss(X, z, pi, mu, cov, lam=lam))

optimizer = optim.SGD([pi, mu, cov], lr=lr)

for step in tqdm(range(max_step)):
    optimizer.zero_grad()
    loss = MoG_loss(X, z, pi, mu, cov, lam=lam)
    loss.backward()
    optimizer.step()

    if (step + 1) % 100 == 0:
        print('Step {}'.format(step))
        print('pi:')
        print(pi)
        print('mu:')
        print(mu)
        print('cov:')
        print(cov)
        print('loss:')
        print(loss)

*** Init ***
pi:
tensor([0.0290, 0.2395, 0.2814, 0.1792, 0.2709], requires_grad=True)
mu:
tensor([[-0.2416,  0.1751],
        [ 1.0049, -0.3738],
        [-0.4084, -1.1477],
        [ 0.4074,  1.6492],
        [-0.0961,  1.1061]], requires_grad=True)
cov:
tensor([[[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]],

        [[1., 0.],
         [0., 1.]]], requires_grad=True)
loss:
tensor(51.8790, grad_fn=<ThSubBackward>)


Step 99
pi:
tensor([nan, nan, nan, nan, nan], requires_grad=True)
mu:
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], requires_grad=True)
cov:
tensor([[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]], requires_grad=True)
loss:
tensor(nan, grad_fn=<ThSubBackward>)
Step 199
pi:
tensor([nan, nan, nan, nan, nan], requires_grad=True)
mu:
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], requires_grad=True)
cov:
tensor([[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]], requires_grad=True)
loss:
tensor(nan, grad_fn=<ThSubBackward>)
Step 299
pi:
tensor([nan, nan, nan, nan, 

KeyboardInterrupt: 